In [48]:
import pandas as pd
from ivpy.cluster import cluster
import shutil
from PIL import Image
import os
import numpy as np
import pickle
import sklearn
from ivpy.cluster import cluster

from annoy import AnnoyIndex


metadata = pd.read_csv("/mnt/e/Tasks/similarity/files/metadata.csv")
feature_matrix = pd.read_pickle("/mnt/e/Tasks/similarity/files/situated_views_features.pickle")
df = pd.read_pickle("/mnt/e/Tasks/similarity/files/situated_views_df.pickle")

#inside bigger folder

In [ ]:
method='K-means'
DIR = "/mnt/e/Tasks/Vistas Situadas - Imagens/classification_gray"
folders = os.listdir(DIR)

for folder in folders:
    folder_list = os.listdir(DIR+folder)    
    index_list = [df["id"].to_list().index(item.split(".")[0])for item in folder_list if item.split(".")[0] in df["id"].to_list()]

    ks = [12,35,5,10]
    df['cluster_{0}'.format(folder)] = cluster(feature_matrix.iloc[index_list],k=ks[folders.index(folder)])
    df2 = df.dropna(subset=['cluster_{0}'.format(folder)])

    for i, item in df2.iterrows(): 
        path_to = "{0}/{1}/{2}/{3}".format(DIR,folder,method,item['cluster_{0}'.format(folder)])
        if item['cluster_{0}'.format(folder)] != np.nan:
            if not os.path.exists(path_to):
                os.makedirs(path_to)
            shutil.copy(item["filename"],path_to+"/"+item["id"]+".jpg") 

#Cluster in clustered folders

In [49]:
def find_nearest_cluster(X, set_img, metadata_df):
    t = AnnoyIndex(feature_matrix.shape[1])
    counter = -1
    for j in X.index:
        counter+=1
        t.add_item(counter,list(X.loc[j]))

    idmap = dict(zip(X.index,list(range(len(X)))))
    idmapReverse = dict(zip(list(range(len(X))),X.index))

    t.build(10) # 10 trees
    items = {}     
    index = set_img.index[0]    
    nn, distances = t.get_nns_by_item(idmap[index],200,include_distances=True)
    nnsAnnoy = dict(zip(nn, distances)) 
    selected = {df.iloc[idmapReverse[item]].id : '{:.3f}'.format(round(nnsAnnoy[item], 3))   for item in nnsAnnoy}                

    return selected

In [ ]:
clusters_affinity50 = "/mnt/e/Tasks/Vistas Situadas - Imagens/classification/affinity_-50"
clusters_affinity100 = "/mnt/e/Tasks/Vistas Situadas - Imagens/classification_gray/affinity_-100"

def order_cluster_nearest(clusters, X, df): 
    for folder in os.listdir(clusters):
        folder_list = [x.split(".")[0] for x in os.listdir(clusters+"/"+folder)]
        df_folder = df.loc[df["id"].isin(folder_list)]
        X_folder = X.iloc[df_folder.index]        
        items = find_nearest_cluster(X_folder, df_folder,df)
       
        for key in items:
            path_to = "{0}_{1}/{2}".format(clusters,"ordered-ne",folder)            
            if not os.path.exists(path_to):
                os.makedirs(path_to)
            try:
                shutil.copy(os.path.join(clusters,folder,key+".jpg"),"{0}/{1}_{2}.jpg".format(path_to, items[key],key))
                #print(os.path.join(clusters,folder,key+".jpg"),"{0}/{1}_{2}.jpg".format(path_to, items[key],key))
            except Exception as e:
                print(key, " Error: ", e)
                pass
            
order_cluster_nearest(clusters_affinity100,feature_matrix,df)

In [52]:
items

{'001AAN005121_geo': '0.000',
 '001AAN005149_geo': '0.658',
 '007A5P3F13-022_geo': '0.667',
 '001AAN005151_geo': '0.689',
 '001AN01002021_geo': '0.713',
 '0071824cx001-12_geo': '0.718',
 '001AAN005147_geo': '0.721',
 '0071824cx001-09_geo': '0.753',
 '002080RJ2202_geo': '0.754',
 '0071824cx046-05_geo': '0.762',
 '002080RJ0103_geo': '0.766',
 '001ANV022024_geo': '0.769',
 '002080BR0215_geo': '0.785',
 '001AN01002019_geo': '0.806',
 '0071824cx036a-09_geo': '0.810',
 '0071824cxrio3-01_geo': '0.822',
 '0073040avulsoscx001e050-01_geo': '0.822',
 '001AN01002020_geo': '0.824',
 '0071824cx001-08_geo': '0.825',
 '0072430cx036-10_geo': '0.829',
 '002080RJN0201_geo': '0.841',
 '002080RJ0104_geo': '0.845',
 '007AFA01_geo': '0.845',
 '002086AMF12_geo': '0.846',
 '007_IMG_2163': '0.847',
 '0072137cx016-05_geo': '0.848',
 '0073040cx026-03_geo': '0.849',
 '0071824cx063-02_geo': '0.850',
 '0072137cx006-10_geo': '0.852',
 '007A5P3F13-037_geo': '0.854',
 '013RJ001003_geo': '0.857',
 '002095RJ001034_geo': 

In [ ]:
DIR = "/mnt/e/Tasks/Vistas Situadas - Imagens/classification/"
cluster_folder = "duplicates_geo"
method='affinity'
#affinity = 'precomputed'
karg = -50
def order_cluster_method(DIR, X, df): 
    cluster_number = 'cluster_{0}'.format(method)   
    for folder in os.listdir(DIR+cluster_folder):        
        df2 = df.copy()
        folder_list = [x.split(".")[0] for x in os.listdir(DIR+cluster_folder+"/"+folder)]
        index_list = [df["id"].to_list().index(item.split(".")[0])for item in folder_list if item.split(".")[0] in df["id"].to_list()]    
       
        print(folder)    
        df2['cluster_{0}'.format(method)] = cluster(X.iloc[index_list],method=method, preference=karg)
        df2 = df2.dropna(subset=['cluster_{0}'.format(method) ])

        for i, item in df2.iterrows(): 
            path_to = "{0}/{1}/{2}".format(DIR,cluster_folder+"_ordered-50",folder)
            if item[cluster_number] != np.nan:
                if not os.path.exists(path_to):
                    os.makedirs(path_to)
                shutil.copy(item["filename"],path_to+"/"+str(item['cluster_{0}'.format(method) ])+"_"+ item["id"]+".jpg")

            
order_cluster_method(DIR,feature_matrix,df)

In [ ]:
DIR = "/mnt/e/Tasks/Vistas Situadas - Imagens/images_cropped/"
method='affinity'
karg = -100

folder_list = os.listdir(DIR)    
index_list = [df["id"].to_list().index(item.split(".")[0])for item in folder_list if item.split(".")[0] in df["id"].to_list()]

df['cluster_{0}'.format(method)] = cluster(feature_matrix.iloc[index_list],method=method, preference=karg)
df2 = df.dropna(subset=['cluster_{0}'.format(method)])


for i, item in df2.iterrows(): 
    path_to = "{0}/{1}/{2}/{3}".format(DIR,method,method+str(karg),item['cluster_{0}'.format(method)])
    if item['cluster_{0}'.format(method)] != np.nan:
        if not os.path.exists(path_to):
            os.makedirs(path_to)
        shutil.copy(item["filename"],path_to+"/"+item["id"]+".jpg") 

In [ ]:
#separe folder by nearest
import re
folder = "/mnt/e/Tasks/Vistas Situadas - Imagens/images_selected"
path_to = "/mnt/e/Tasks/Vistas Situadas - Imagens/images_selected_ordered_ne2"
folder_code = os.listdir(folder)

folder_list = [(re.search(r"\.0_(.*).jpg", x)).group(1) for x in os.listdir(folder)]
processed_ = {}
for identifier in folder_list: 
    #print(identifier)
    for i,item in enumerate(duplicates):
        index = False
        if item[0] == identifier:            
            if not identifier in processed_.keys(): 
                for img in item[1]:                                    
                    processed_[img] = i
            else:
                processed_[identifier] = 5000

for key in processed_:
    name = folder_code[folder_list.index(key)]
    print(name, processed_[key])
    shutil.copy("{0}/{1}".format(folder,name),"{0}/{1}_{2}".format(path_to, processed_[key] ,name)) 